**Mô hình dự đoán giá cổ phiếu**

Kết nối với dữ liệu từ Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Gọi những thư viện cần thiết

In [ ]:
import warnings
import pandas as pd
import numpy as np
from numpy import array
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.pandas.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

Đọc các File báo cáo giá cổ phiếu của FPT, MSN, PNJ, và VIC

In [ ]:
file_path_fpt = '/content/drive/MyDrive/WORK/Finpros/FPT.csv'
df_fpt = pd.read_csv(file_path_fpt)
file_path_msn = '/content/drive/MyDrive/WORK/Finpros/MSN.csv'
df_msn = pd.read_csv(file_path_msn)
file_path_pnj = '/content/drive/MyDrive/WORK/Finpros/PNJ.csv'
df_pnj = pd.read_csv(file_path_pnj)
file_path_vic = '/content/drive/MyDrive/WORK/Finpros/VIC.csv'
df_vic = pd.read_csv(file_path_vic)

In [ ]:
df_fpt

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest
0,FPT,12/25/2018 9:15,30.89,30.89,30.89,30.89,35410,0
1,FPT,12/25/2018 9:16,30.81,30.81,30.81,30.81,190,0
2,FPT,12/25/2018 9:17,30.74,30.81,30.74,30.74,1120,0
3,FPT,12/25/2018 9:18,30.74,30.74,30.74,30.74,2120,0
4,FPT,12/25/2018 9:19,30.74,30.74,30.74,30.74,22500,0
...,...,...,...,...,...,...,...,...
97401,FPT,12/22/2020 14:24,58.20,58.20,58.20,58.20,27470,0
97402,FPT,12/22/2020 14:25,58.20,58.20,58.20,58.20,14820,0
97403,FPT,12/22/2020 14:26,58.20,58.20,58.20,58.20,500,0
97404,FPT,12/22/2020 14:29,58.10,58.10,58.10,58.10,2500,0


In [ ]:
df_msn

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest
0,MSN,12/25/2017 9:15,73.1,73.1,73.1,73.1,4210,0
1,MSN,12/25/2017 9:16,73.0,73.0,73.0,73.0,5000,0
2,MSN,12/25/2017 9:18,73.5,73.5,73.5,73.5,210,0
3,MSN,12/25/2017 9:20,73.2,73.5,73.1,73.1,2050,0
4,MSN,12/25/2017 9:21,73.0,73.0,73.0,73.0,1380,0
...,...,...,...,...,...,...,...,...
135349,MSN,12/22/2020 14:23,82.8,82.8,82.8,82.8,700,0
135350,MSN,12/22/2020 14:24,82.7,82.8,82.7,82.8,4660,0
135351,MSN,12/22/2020 14:25,82.8,82.8,82.8,82.8,50,0
135352,MSN,12/22/2020 14:27,82.8,82.8,82.8,82.8,300,0


In [ ]:
df_pnj

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest
0,PNJ,2/28/2018 9:15,78.14,78.99,78.14,78.99,270,0
1,PNJ,2/28/2018 9:16,78.94,78.94,78.94,78.94,10,0
2,PNJ,2/28/2018 9:19,78.14,78.14,78.14,78.14,283,0
3,PNJ,2/28/2018 9:20,78.14,78.14,78.14,78.14,480,0
4,PNJ,2/28/2018 9:21,78.14,78.14,78.14,78.14,146,0
...,...,...,...,...,...,...,...,...
125304,PNJ,12/22/2020 14:22,78.20,78.20,78.20,78.20,5000,0
125305,PNJ,12/22/2020 14:23,78.20,78.20,78.20,78.20,1700,0
125306,PNJ,12/22/2020 14:24,78.20,78.20,78.20,78.20,870,0
125307,PNJ,12/22/2020 14:25,78.10,78.10,78.10,78.10,13000,0


In [ ]:
df_vic

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest
0,VIC,10/17/2018 9:39,98.5,98.5,98.5,98.5,390,0
1,VIC,10/17/2018 9:40,98.5,98.5,98.5,98.5,760,0
2,VIC,10/17/2018 9:41,98.6,98.6,98.6,98.6,2060,0
3,VIC,10/17/2018 9:42,98.6,98.6,98.6,98.6,5540,0
4,VIC,10/17/2018 9:43,98.6,98.7,98.6,98.7,760,0
...,...,...,...,...,...,...,...,...
101261,VIC,12/22/2020 14:22,105.8,105.8,105.7,105.7,1870,0
101262,VIC,12/22/2020 14:23,105.7,105.7,105.7,105.7,2510,0
101263,VIC,12/22/2020 14:24,105.7,105.7,105.7,105.7,3620,0
101264,VIC,12/22/2020 14:25,105.7,105.7,105.7,105.7,100,0


Gom các hàng trong bảng dữ liệu lại theo từng ngày, hiển thị thông tin giá cổ phiếu cao nhất, thấp nhất và thời điểm xuất hiện.

In [ ]:
# Chuyển đổi cột 'Date/Time' thành datetime
df_fpt['Date/Time'] = pd.to_datetime(df_fpt['Date/Time'])
df_msn['Date/Time'] = pd.to_datetime(df_msn['Date/Time'])
df_pnj['Date/Time'] = pd.to_datetime(df_pnj['Date/Time'])
df_vic['Date/Time'] = pd.to_datetime(df_vic['Date/Time'])

# Sau đó thực hiện nhóm và tổng hợp
df_fpt_grouped = df_fpt.groupby([df_fpt['Date/Time'].dt.date, 'Ticker']).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).reset_index()
df_fpt_temp = df_fpt.set_index('Date/Time')
df_fpt_grouped['Time_High'] = df_fpt_temp.groupby([df_fpt_temp.index.date, 'Ticker'])['High'].idxmax().values
df_fpt_grouped['Time_Low'] = df_fpt_temp.groupby([df_fpt_temp.index.date, 'Ticker'])['Low'].idxmin().values
df_fpt_grouped['Time_High'] = df_fpt_grouped['Time_High'].dt.time
df_fpt_grouped['Time_Low'] = df_fpt_grouped['Time_Low'].dt.time

# Process MSN data
df_msn_grouped = df_msn.groupby([df_msn['Date/Time'].dt.date, 'Ticker']).agg({
  'Open': 'first',
  'High': 'max',
  'Low': 'min',
  'Close': 'last',
  'Volume': 'sum'
}).reset_index()
df_msn_temp = df_msn.set_index('Date/Time')
df_msn_grouped['Time_High'] = df_msn_temp.groupby([df_msn_temp.index.date, 'Ticker'])['High'].idxmax().values
df_msn_grouped['Time_Low'] = df_msn_temp.groupby([df_msn_temp.index.date, 'Ticker'])['Low'].idxmin().values
df_msn_grouped['Time_High'] = df_msn_grouped['Time_High'].dt.time
df_msn_grouped['Time_Low'] = df_msn_grouped['Time_Low'].dt.time

# Process VIC data
df_vic_grouped = df_vic.groupby([df_vic['Date/Time'].dt.date, 'Ticker']).agg({
  'Open': 'first',
  'High': 'max',
  'Low': 'min',
  'Close': 'last',
  'Volume': 'sum'
}).reset_index()
df_vic_temp = df_vic.set_index('Date/Time')
df_vic_grouped['Time_High'] = df_vic_temp.groupby([df_vic_temp.index.date, 'Ticker'])['High'].idxmax().values
df_vic_grouped['Time_Low'] = df_vic_temp.groupby([df_vic_temp.index.date, 'Ticker'])['Low'].idxmin().values
df_vic_grouped['Time_High'] = df_vic_grouped['Time_High'].dt.time
df_vic_grouped['Time_Low'] = df_vic_grouped['Time_Low'].dt.time

# Process PNJ data
df_pnj_grouped = df_pnj.groupby([df_pnj['Date/Time'].dt.date, 'Ticker']).agg({
  'Open': 'first',
  'High': 'max',
  'Low': 'min',
  'Close': 'last',
  'Volume': 'sum'
}).reset_index()
df_pnj_temp = df_pnj.set_index('Date/Time')
df_pnj_grouped['Time_High'] = df_pnj_temp.groupby([df_pnj_temp.index.date, 'Ticker'])['High'].idxmax().values
df_pnj_grouped['Time_Low'] = df_pnj_temp.groupby([df_pnj_temp.index.date, 'Ticker'])['Low'].idxmin().values
df_pnj_grouped['Time_High'] = df_pnj_grouped['Time_High'].dt.time
df_pnj_grouped['Time_Low'] = df_pnj_grouped['Time_Low'].dt.time

In [ ]:
df_fpt_grouped

,Date/Time,Ticker,Open,High,Low,Close,Volume,Time_High,Time_Low
0,2018-12-25,FPT,30.89,30.89,30.59,30.66,84960,09:15:00,09:21:00
1,2018-12-26,FPT,31.04,31.22,30.89,30.89,333590,09:27:00,09:41:00
2,2018-12-27,FPT,31.41,31.52,31.19,31.19,454860,09:15:00,09:17:00
3,2018-12-28,FPT,31.34,31.49,31.19,31.49,387170,14:46:00,09:41:00
4,2019-01-02,FPT,31.19,31.45,30.70,30.74,459120,09:18:00,14:29:00
...,...,...,...,...,...,...,...,...,...
494,2020-12-16,FPT,57.20,57.50,57.00,57.10,1995530,10:18:00,09:17:00
495,2020-12-17,FPT,56.90,57.00,56.00,56.50,3594500,09:22:00,14:00:00
496,2020-12-18,FPT,56.50,57.00,56.50,56.90,1532820,09:16:00,09:15:00
497,2020-12-21,FPT,57.30,57.80,57.10,57.50,2307390,11:03:00,09:34:00


In [ ]:
df_msn_grouped

,Date/Time,Ticker,Open,High,Low,Close,Volume,Time_High,Time_Low
0,2017-12-25,MSN,73.10,75.4,73.00,74.2,207370,14:19:00,09:16:00
1,2017-12-26,MSN,74.90,75.5,74.30,75.5,403710,09:19:00,09:51:00
2,2017-12-27,MSN,74.90,75.7,74.80,75.0,517920,10:47:00,09:19:00
3,2017-12-28,MSN,75.00,76.0,74.50,75.7,377030,13:55:00,09:25:00
4,2017-12-29,MSN,76.00,77.0,76.00,76.7,226880,14:12:00,13:35:00
...,...,...,...,...,...,...,...,...,...
743,2020-12-16,MSN,83.01,83.6,82.91,83.5,648540,10:12:00,09:20:00
744,2020-12-17,MSN,83.50,84.3,82.90,84.0,1179730,13:38:00,09:40:00
745,2020-12-18,MSN,83.70,84.3,83.70,84.1,1417470,11:13:00,09:15:00
746,2020-12-21,MSN,85.00,85.0,83.40,83.8,827020,09:15:00,14:22:00


In [ ]:
df_pnj_grouped

,Date/Time,Ticker,Open,High,Low,Close,Volume,Time_High,Time_Low
0,2018-02-28,PNJ,78.14,80.61,78.09,78.71,8158,09:38:00,09:22:00
1,2018-03-01,PNJ,78.14,80.32,77.66,77.66,20511,10:00:00,14:13:00
2,2018-03-02,PNJ,77.14,77.90,76.86,77.33,37274,13:17:00,10:39:00
3,2018-03-05,PNJ,77.14,80.50,77.14,78.28,27737,13:31:00,09:15:00
4,2018-03-06,PNJ,78.61,83.13,77.43,81.45,41077,14:28:00,09:33:00
...,...,...,...,...,...,...,...,...,...
704,2020-12-16,PNJ,77.20,77.60,77.00,77.60,636190,13:22:00,10:20:00
705,2020-12-17,PNJ,76.50,77.40,75.00,75.80,1289500,09:41:00,14:01:00
706,2020-12-18,PNJ,75.80,77.50,75.80,77.30,582740,09:47:00,09:15:00
707,2020-12-21,PNJ,77.30,77.40,76.50,77.30,676290,10:24:00,13:10:00


In [ ]:
df_vic_grouped

,Date/Time,Ticker,Open,High,Low,Close,Volume,Time_High,Time_Low
0,2018-10-17,VIC,98.5,99.6,98.3,99.0,1266610,10:05:00,11:06:00
1,2018-10-18,VIC,98.5,99.2,98.0,99.1,896140,14:00:00,09:18:00
2,2018-10-19,VIC,96.5,99.1,96.5,99.1,915870,14:46:00,09:15:00
3,2018-10-22,VIC,98.1,99.6,97.1,99.1,536930,10:21:00,09:17:00
4,2018-10-23,VIC,98.1,99.0,96.5,99.0,608750,09:15:00,13:09:00
...,...,...,...,...,...,...,...,...,...
543,2020-12-16,VIC,106.8,107.1,105.9,107.0,756200,09:30:00,10:27:00
544,2020-12-17,VIC,106.3,106.9,104.7,104.7,1352580,09:15:00,10:24:00
545,2020-12-18,VIC,104.7,106.1,104.5,104.7,1350780,11:20:00,09:15:00
546,2020-12-21,VIC,105.5,106.4,104.9,106.4,587100,14:46:00,09:54:00


**Hiển thị đồ thị nến với đường Bollinger của từng mã cổ phiếu để theo dõi biến động và tạo đồ thị thể hiện sự thay đổi của giá cổ phiếu và khối lượng giao dịch**

Cổ phiếu FPT

In [ ]:
# Filter data for the specific ticker
dfg_fpt = df_fpt_grouped[df_fpt_grouped['Ticker'] == 'FPT']

# Create a custom hover text
hover_text = []
for index, row in dfg_fpt.iterrows():
    hover_text.append(('Date/Time: {}<br>' +
                       'Open: {}<br>' +
                       'High: {}<br>' +
                       'Time of High: {}<br>' +
                       'Low: {}<br>' +
                       'Time of High: {}<br>' +
                       'Close: {}').format(row['Date/Time'],
                                           row['Open'],
                                           row['High'],
                                           row['Time_High'],
                                           row['Low'],
                                           row['Time_Low'],
                                           row['Close']))

fig = go.Figure(data=[go.Candlestick(x=dfg_fpt['Date/Time'],
                                     open=dfg_fpt['Open'],
                                     high=dfg_fpt['High'],
                                     low=dfg_fpt['Low'],
                                     close=dfg_fpt['Close'],
                                     text=hover_text,
                                     hoverinfo='text')])

# Calculate moving average and standard deviation
dfg_fpt['MA'] = dfg_fpt['Close'].rolling(window=20).mean()
dfg_fpt['STD'] = dfg_fpt['Close'].rolling(window=20).std()

# Calculate Bollinger Bands
dfg_fpt['Upper_Band'] = dfg_fpt['MA'] + (dfg_fpt['STD'] * 2)
dfg_fpt['Lower_Band'] = dfg_fpt['MA'] - (dfg_fpt['STD'] * 2)

fig.add_trace(go.Scatter(x=dfg_fpt['Date/Time'], y=dfg_fpt['Upper_Band'], name='Upper Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Scatter(x=dfg_fpt['Date/Time'], y=dfg_fpt['Lower_Band'], name='Lower Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Candlestick(x=dfg_fpt['Date/Time'], open=dfg_fpt['Open'], high=dfg_fpt['High'], low=dfg_fpt['Low'], close=dfg_fpt['Close'], name='Candlestick'))

fig.show()


In [ ]:
# Create a figure
fig = go.Figure()

# Add 'Close' trace
fig.add_trace(go.Scatter(x=dfg_fpt['Date/Time'], y=dfg_fpt['Close'], mode='lines', name='Close'))

# Add 'Volume' trace
fig.add_trace(go.Scatter(x=dfg_fpt['Date/Time'], y=dfg_fpt['Volume'], mode='lines', name='Volume', yaxis='y2'))

# Create a second y-axis for 'Volume'
fig.update_layout(yaxis2=dict(overlaying='y', side='right'))

fig.show()

Cổ phiếu MSN

In [ ]:
# Filter data for the specific ticker
dfg_msn = df_msn_grouped[df_msn_grouped['Ticker'] == 'MSN']

# Create a custom hover text
hover_text = []
for index, row in dfg_msn.iterrows():
  hover_text.append(('Date/Time: {}<br>' +
                     'Open: {}<br>' +
                     'High: {}<br>' +
                     'Time of High: {}<br>' +
                     'Low: {}<br>' +
                     'Time of High: {}<br>' +
                     'Close: {}').format(row['Date/Time'],
                                       row['Open'],
                                       row['High'],
                                       row['Time_High'],
                                       row['Low'],
                                       row['Time_Low'],
                                       row['Close']))

fig = go.Figure(data=[go.Candlestick(x=dfg_msn['Date/Time'],
                                     open=dfg_msn['Open'],
                                     high=dfg_msn['High'],
                                     low=dfg_msn['Low'],
                                     close=dfg_msn['Close'],
                                     text=hover_text,
                                     hoverinfo='text')])

# Calculate moving average and standard deviation
dfg_msn['MA'] = dfg_msn['Close'].rolling(window=20).mean()
dfg_msn['STD'] = dfg_msn['Close'].rolling(window=20).std()

# Calculate Bollinger Bands
dfg_msn['Upper_Band'] = dfg_msn['MA'] + (dfg_msn['STD'] * 2)
dfg_msn['Lower_Band'] = dfg_msn['MA'] - (dfg_msn['STD'] * 2)

fig.add_trace(go.Scatter(x=dfg_msn['Date/Time'], y=dfg_msn['Upper_Band'], name='Upper Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Scatter(x=dfg_msn['Date/Time'], y=dfg_msn['Lower_Band'], name='Lower Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Candlestick(x=dfg_msn['Date/Time'], open=dfg_msn['Open'], high=dfg_msn['High'], low=dfg_msn['Low'], close=dfg_msn['Close'], name='Candlestick'))

fig.show()


In [ ]:
# Create a figure
fig = go.Figure()

# Add 'Close' trace
fig.add_trace(go.Scatter(x=dfg_msn['Date/Time'], y=dfg_msn['Close'], mode='lines', name='Close'))

# Add 'Volume' trace
fig.add_trace(go.Scatter(x=dfg_msn['Date/Time'], y=dfg_msn['Volume'], mode='lines', name='Volume', yaxis='y2'))

# Create a second y-axis for 'Volume'
fig.update_layout(yaxis2=dict(overlaying='y', side='right'))

fig.show()

Cổ phiếu PNJ

In [ ]:
# Filter data for the specific ticker
dfg_pnj = df_pnj_grouped[df_pnj_grouped['Ticker'] == 'PNJ']

# Create a custom hover text
hover_text = []
for index, row in dfg_pnj.iterrows():
  hover_text.append(('Date/Time: {}<br>' +
                     'Open: {}<br>' +
                     'High: {}<br>' +
                     'Time of High: {}<br>' +
                     'Low: {}<br>' +
                     'Time of High: {}<br>' +
                     'Close: {}').format(row['Date/Time'],
                                       row['Open'],
                                       row['High'],
                                       row['Time_High'],
                                       row['Low'],
                                       row['Time_Low'],
                                       row['Close']))

fig = go.Figure(data=[go.Candlestick(x=dfg_pnj['Date/Time'],
                                     open=dfg_pnj['Open'],
                                     high=dfg_pnj['High'],
                                     low=dfg_pnj['Low'],
                                     close=dfg_pnj['Close'],
                                     text=hover_text,
                                     hoverinfo='text')])

# Calculate moving average and standard deviation
dfg_pnj['MA'] = dfg_pnj['Close'].rolling(window=20).mean()
dfg_pnj['STD'] = dfg_pnj['Close'].rolling(window=20).std()

# Calculate Bollinger Bands
dfg_pnj['Upper_Band'] = dfg_pnj['MA'] + (dfg_pnj['STD'] * 2)
dfg_pnj['Lower_Band'] = dfg_pnj['MA'] - (dfg_pnj['STD'] * 2)

fig.add_trace(go.Scatter(x=dfg_pnj['Date/Time'], y=dfg_pnj['Upper_Band'], name='Upper Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Scatter(x=dfg_pnj['Date/Time'], y=dfg_pnj['Lower_Band'], name='Lower Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Candlestick(x=dfg_pnj['Date/Time'], open=dfg_pnj['Open'], high=dfg_pnj['High'], low=dfg_pnj['Low'], close=dfg_pnj['Close'], name='Candlestick'))

fig.show()

In [ ]:
# Create a figure
fig = go.Figure()

# Add 'Close' trace
fig.add_trace(go.Scatter(x=dfg_pnj['Date/Time'], y=dfg_pnj['Close'], mode='lines', name='Close'))

# Add 'Volume' trace
fig.add_trace(go.Scatter(x=dfg_pnj['Date/Time'], y=dfg_pnj['Volume'], mode='lines', name='Volume', yaxis='y2'))

# Create a second y-axis for 'Volume'
fig.update_layout(yaxis2=dict(overlaying='y', side='right'))

fig.show()

Cổ phiếu VIC

In [ ]:
# Filter data for the specific ticker
dfg_vic = df_vic_grouped[df_vic_grouped['Ticker'] == 'VIC']

# Create a custom hover text
hover_text = []
for index, row in dfg_vic.iterrows():
  hover_text.append(('Date/Time: {}<br>' +
                     'Open: {}<br>' +
                     'High: {}<br>' +
                     'Time of High: {}<br>' +
                     'Low: {}<br>' +
                     'Time of High: {}<br>' +
                     'Close: {}').format(row['Date/Time'],
                                       row['Open'],
                                       row['High'],
                                       row['Time_High'],
                                       row['Low'],
                                       row['Time_Low'],
                                       row['Close']))

fig = go.Figure(data=[go.Candlestick(x=dfg_vic['Date/Time'],
                                     open=dfg_vic['Open'],
                                     high=dfg_vic['High'],
                                     low=dfg_vic['Low'],
                                     close=dfg_vic['Close'],
                                     text=hover_text,
                                     hoverinfo='text')])

# Calculate moving average and standard deviation
dfg_vic['MA'] = dfg_vic['Close'].rolling(window=20).mean()
dfg_vic['STD'] = dfg_vic['Close'].rolling(window=20).std()

# Calculate Bollinger Bands
dfg_vic['Upper_Band'] = dfg_vic['MA'] + (dfg_vic['STD'] * 2)
dfg_vic['Lower_Band'] = dfg_vic['MA'] - (dfg_vic['STD'] * 2)

fig.add_trace(go.Scatter(x=dfg_vic['Date/Time'], y=dfg_vic['Upper_Band'], name='Upper Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Scatter(x=dfg_vic['Date/Time'], y=dfg_vic['Lower_Band'], name='Lower Band', line=dict(color='rgba(255,165,0, 0.5)')))
fig.add_trace(go.Candlestick(x=dfg_vic['Date/Time'], open=dfg_vic['Open'], high=dfg_vic['High'], low=dfg_vic['Low'], close=dfg_vic['Close'], name='Candlestick'))

fig.show()


In [ ]:
# Create a figure
fig = go.Figure()

# Add 'Close' trace
fig.add_trace(go.Scatter(x=dfg_vic['Date/Time'], y=dfg_vic['Close'], mode='lines', name='Close'))

# Add 'Volume' trace
fig.add_trace(go.Scatter(x=dfg_vic['Date/Time'], y=dfg_vic['Volume'], mode='lines', name='Volume', yaxis='y2'))

# Create a second y-axis for 'Volume'
fig.update_layout(yaxis2=dict(overlaying='y', side='right'))

fig.show()

Phân tích : Dựa vào tỷ giá thay đổi vào từng thời điểm trong ngày, giả định giá giảm vì cổ phiếu được bán ra nhiều và giá tăng khi cổ phiếu được mua vào nhiều, vậy có thể xây dựng bảng dữ liệu khối lượng mua và bán của mã cổ phiếu trong ngày

In [ ]:
# Create a new DataFrame with only the required columns
df_fpt_Volume_open = df_fpt[['Date/Time', 'Volume', 'Open']].copy()
df_msn_Volume_open = df_msn[['Date/Time', 'Volume', 'Open']].copy()
df_pnj_Volume_open = df_pnj[['Date/Time', 'Volume', 'Open']].copy()
df_vic_Volume_open = df_vic[['Date/Time', 'Volume', 'Open']].copy()

# Create a new column 'Group' that increments each time 'Open' changes
df_fpt_Volume_open['Group'] = (df_fpt_Volume_open['Open'] != df_fpt_Volume_open['Open'].shift()).cumsum()
df_msn_Volume_open['Group'] = (df_msn_Volume_open['Open'] != df_msn_Volume_open['Open'].shift()).cumsum()
df_pnj_Volume_open['Group'] = (df_pnj_Volume_open['Open'] != df_pnj_Volume_open['Open'].shift()).cumsum()
df_vic_Volume_open['Group'] = (df_vic_Volume_open['Open'] != df_vic_Volume_open['Open'].shift()).cumsum()

# Group by 'Group' and aggregate
df_fpt_grouped2 = df_fpt_Volume_open.groupby('Group').agg({
    'Open': 'first',
    'Date/Time' : 'first',
    'Volume': 'sum'
}).reset_index(drop=True)

df_msn_grouped2 = df_msn_Volume_open.groupby('Group').agg({
    'Open': 'first',
    'Date/Time' : 'first',
    'Volume': 'sum'
}).reset_index(drop=True)

df_pnj_grouped2 = df_pnj_Volume_open.groupby('Group').agg({
    'Open': 'first',
    'Date/Time' : 'first',
    'Volume': 'sum'
}).reset_index(drop=True)

df_vic_grouped2 = df_vic_Volume_open.groupby('Group').agg({
    'Open': 'first',
    'Date/Time' : 'first',
    'Volume': 'sum'
}).reset_index(drop=True)

df_fpt_grouped2['Prev_Open'] = df_fpt_grouped2['Open'].shift(-1)
df_fpt_grouped2.loc[df_fpt_grouped2['Open'] > df_fpt_grouped2['Prev_Open'], 'Volume'] *= -1
df_fpt_grouped2.drop('Prev_Open', axis=1, inplace=True)

df_msn_grouped2['Prev_Open'] = df_msn_grouped2['Open'].shift(-1)
df_msn_grouped2.loc[df_msn_grouped2['Open'] > df_msn_grouped2['Prev_Open'], 'Volume'] *= -1
df_msn_grouped2.drop('Prev_Open', axis=1, inplace=True)

df_pnj_grouped2['Prev_Open'] = df_pnj_grouped2['Open'].shift(-1)
df_pnj_grouped2.loc[df_pnj_grouped2['Open'] > df_pnj_grouped2['Prev_Open'], 'Volume'] *= -1
df_pnj_grouped2.drop('Prev_Open', axis=1, inplace=True)

df_vic_grouped2['Prev_Open'] = df_vic_grouped2['Open'].shift(-1)
df_vic_grouped2.loc[df_vic_grouped2['Open'] > df_vic_grouped2['Prev_Open'], 'Volume'] *= -1
df_vic_grouped2.drop('Prev_Open', axis=1, inplace=True)

Dữ liệu về khối lượng mua và bán của mã cổ phiếu FPT

In [ ]:
# Create 'Sell' and 'Sold' columns
df_fpt_grouped2['Sold'] = df_fpt_grouped2['Volume'].apply(lambda x: x if x < 0 else 0)
df_fpt_grouped2['Bought'] = df_fpt_grouped2['Volume'].apply(lambda x: x if x > 0 else 0)
df_fpt_grouped2['Date/Time'] = pd.to_datetime(df_fpt_grouped2['Date/Time']).dt.date
df_fpt_grouped2 = df_fpt_grouped2.groupby('Date/Time').agg({
    'Sold': 'sum',
    'Bought': 'sum'
}).reset_index()
df_fpt_grouped2

,Date/Time,Sold,Bought
0,2018-12-25,-68480,16480
1,2018-12-26,-178340,155250
2,2018-12-27,-195240,259620
3,2018-12-28,-266090,121080
4,2019-01-02,-224170,234950
...,...,...,...
494,2020-12-16,-1217270,778260
495,2020-12-17,-2046760,1603060
496,2020-12-18,-797620,679880
497,2020-12-21,-1141350,1166040


Dữ liệu về khối lượng mua và bán của mã cổ phiếu MSN

In [ ]:
# Create 'Sell' and 'Sold' columns
df_msn_grouped2['Sold'] = df_msn_grouped2['Volume'].apply(lambda x: x if x < 0 else 0)
df_msn_grouped2['Bought'] = df_msn_grouped2['Volume'].apply(lambda x: x if x > 0 else 0)
df_msn_grouped2['Date/Time'] = pd.to_datetime(df_msn_grouped2['Date/Time']).dt.date
df_msn_grouped2 = df_msn_grouped2.groupby('Date/Time').agg({
    'Sold': 'sum',
    'Bought': 'sum'
}).reset_index()
df_msn_grouped2

,Date/Time,Sold,Bought
0,2017-12-25,-68070,139300
1,2017-12-26,-227760,175950
2,2017-12-27,-296160,229300
3,2017-12-28,-75710,293780
4,2017-12-29,-78910,147970
...,...,...,...
743,2020-12-16,-396330,276050
744,2020-12-17,-376590,779300
745,2020-12-18,-362720,1054750
746,2020-12-21,-491540,341790


Dữ liệu về khối lượng mua và bán của mã cổ phiếu PNJ

In [ ]:
# Create 'Sell' and 'Sold' columns
df_pnj_grouped2['Sold'] = df_pnj_grouped2['Volume'].apply(lambda x: x if x < 0 else 0)
df_pnj_grouped2['Bought'] = df_pnj_grouped2['Volume'].apply(lambda x: x if x > 0 else 0)
df_pnj_grouped2['Date/Time'] = pd.to_datetime(df_pnj_grouped2['Date/Time']).dt.date
df_pnj_grouped2 = df_pnj_grouped2.groupby('Date/Time').agg({
    'Sold': 'sum',
    'Bought': 'sum'
}).reset_index()
df_pnj_grouped2

,Date/Time,Sold,Bought
0,2018-02-28,-4125,4033
1,2018-03-01,-11339,9172
2,2018-03-02,-22927,14347
3,2018-03-05,-9313,18424
4,2018-03-06,-9672,31405
...,...,...,...
704,2020-12-16,-290280,345910
705,2020-12-17,-753000,558860
706,2020-12-18,-270550,300790
707,2020-12-21,-249030,416300


Dữ liệu về khối lượng mua và bán của mã cổ phiếu VIC

In [ ]:
# Create 'Sell' and 'Sold' columns
df_vic_grouped2['Sold'] = df_vic_grouped2['Volume'].apply(lambda x: x if x < 0 else 0)
df_vic_grouped2['Bought'] = df_vic_grouped2['Volume'].apply(lambda x: x if x > 0 else 0)
df_vic_grouped2['Date/Time'] = pd.to_datetime(df_vic_grouped2['Date/Time']).dt.date
df_vic_grouped2 = df_vic_grouped2.groupby('Date/Time').agg({
    'Sold': 'sum',
    'Bought': 'sum'
}).reset_index()
df_vic_grouped2

,Date/Time,Sold,Bought
0,2018-10-17,-817510,449100
1,2018-10-18,-441630,454510
2,2018-10-19,-662900,252970
3,2018-10-22,-313130,223800
4,2018-10-23,-300000,308750
...,...,...,...
543,2020-12-16,-485180,271020
544,2020-12-17,-835410,576190
545,2020-12-18,-358140,933620
546,2020-12-21,-382310,204790


**Xây dựng bảng dữ liệu về phần trăm thay đổi của giá cổ phiếu và khối lượng mua bán theo ngày**

Phần trăm thay đổi về khối lượng mua bán và giá cổ phiếu của FPT theo ngày

In [ ]:
# Filter data for the specific ticker
df_fpt_3 = df_fpt_grouped[df_fpt_grouped['Ticker'] == 'FPT']

# Calculate percentage change for 'Close' and 'Volume'
df_fpt_3['Pct_Change_Price'] = df_fpt_3['Close'].pct_change()
df_fpt_3['Pct_Change_Volume'] = df_fpt_3['Volume'].pct_change()

# Create a new DataFrame with only the required columns
df_fpt_pct = df_fpt_3[['Date/Time', 'Pct_Change_Price', 'Pct_Change_Volume']].copy()

# Display the new DataFrame
print(df_fpt_pct)

      Date/Time  Pct_Change_Price  Pct_Change_Volume
0    2018-12-25               NaN                NaN
1    2018-12-26          0.007502           2.926436
2    2018-12-27          0.009712           0.363530
3    2018-12-28          0.009618          -0.148815
4    2019-01-02         -0.023817           0.185836
..          ...               ...                ...
494  2020-12-16          0.007055          -0.251742
495  2020-12-17         -0.010508           0.801276
496  2020-12-18          0.007080          -0.573565
497  2020-12-21          0.010545           0.505324
498  2020-12-22          0.010435           0.116582

[499 rows x 3 columns]


Phần trăm thay đổi về khối lượng mua bán và giá cổ phiếu của MSN theo ngày

In [ ]:
# Filter data for the specific ticker
df_msn_3 = df_msn_grouped[df_msn_grouped['Ticker'] == 'MSN']

# Calculate percentage change for 'Close' and 'Volume'
df_msn_3['Pct_Change_Price'] = df_msn_3['Close'].pct_change()
df_msn_3['Pct_Change_Volume'] = df_msn_3['Volume'].pct_change()

# Create a new DataFrame with only the required columns
df_msn_pct = df_msn_3[['Date/Time', 'Pct_Change_Price', 'Pct_Change_Volume']].copy()

# Display the new DataFrame
print(df_msn_pct)

      Date/Time  Pct_Change_Price  Pct_Change_Volume
0    2017-12-25               NaN                NaN
1    2017-12-26          0.017520           0.946810
2    2017-12-27         -0.006623           0.282901
3    2017-12-28          0.009333          -0.272030
4    2017-12-29          0.013210          -0.398244
..          ...               ...                ...
743  2020-12-16          0.000000          -0.306382
744  2020-12-17          0.005988           0.819055
745  2020-12-18          0.001190           0.201521
746  2020-12-21         -0.003567          -0.416552
747  2020-12-22         -0.011933          -0.026323

[748 rows x 3 columns]


Phần trăm thay đổi về khối lượng mua bán và giá cổ phiếu của PNJ theo ngày

In [ ]:
# Filter data for the specific ticker
df_pnj_3 = df_pnj_grouped[df_pnj_grouped['Ticker'] == 'PNJ']

# Calculate percentage change for 'Close' and 'Volume'
df_pnj_3['Pct_Change_Price'] = df_pnj_3['Close'].pct_change()
df_pnj_3['Pct_Change_Volume'] = df_pnj_3['Volume'].pct_change()

# Create a new DataFrame with only the required columns
df_pnj_pct = df_pnj_3[['Date/Time', 'Pct_Change_Price', 'Pct_Change_Volume']].copy()

# Display the new DataFrame
print(df_pnj_pct)

      Date/Time  Pct_Change_Price  Pct_Change_Volume
0    2018-02-28               NaN                NaN
1    2018-03-01         -0.013340           1.514219
2    2018-03-02         -0.004249           0.817269
3    2018-03-05          0.012285          -0.255862
4    2018-03-06          0.040496           0.480946
..          ...               ...                ...
704  2020-12-16          0.006485          -0.266893
705  2020-12-17         -0.023196           1.026910
706  2020-12-18          0.019789          -0.548088
707  2020-12-21          0.000000           0.160535
708  2020-12-22          0.012937           0.415192

[709 rows x 3 columns]


Phần trăm thay đổi về khối lượng mua bán và giá cổ phiếu của VIC theo ngày

In [ ]:
# Filter data for the specific ticker
df_vic_3 = df_vic_grouped[df_vic_grouped['Ticker'] == 'VIC']

# Calculate percentage change for 'Close' and 'Volume'
df_vic_3['Pct_Change_Price'] = df_vic_3['Close'].pct_change()
df_vic_3['Pct_Change_Volume'] = df_vic_3['Volume'].pct_change()

# Create a new DataFrame with only the required columns
df_vic_pct = df_vic_3[['Date/Time', 'Pct_Change_Price', 'Pct_Change_Volume']].copy()

# Display the new DataFrame
print(df_vic_pct)

      Date/Time  Pct_Change_Price  Pct_Change_Volume
0    2018-10-17               NaN                NaN
1    2018-10-18          0.001010          -0.292489
2    2018-10-19          0.000000           0.022017
3    2018-10-22          0.000000          -0.413749
4    2018-10-23         -0.001009           0.133760
..          ...               ...                ...
543  2020-12-16          0.000000          -0.372812
544  2020-12-17         -0.021495           0.788654
545  2020-12-18          0.000000          -0.001331
546  2020-12-21          0.016237          -0.565362
547  2020-12-22         -0.006579          -0.117152

[548 rows x 3 columns]


In [349]:
# Convert 'Date/Time' to datetime in both DataFrames
df_fpt_pct['Date/Time'] = pd.to_datetime(df_fpt_pct['Date/Time'])
df_fpt_grouped2['Date/Time'] = pd.to_datetime(df_fpt_grouped2['Date/Time'])

# Merge the two DataFrames on 'Date/Time'
df_fpt_merged = pd.merge(df_fpt_pct, df_fpt_grouped2, on='Date/Time', how='inner')
df_fpt_merged['Sold'] = df_fpt_merged['Sold'].abs()

# Convert 'Date/Time' to datetime in both DataFrames
df_msn_pct['Date/Time'] = pd.to_datetime(df_msn_pct['Date/Time'])
df_msn_grouped2['Date/Time'] = pd.to_datetime(df_msn_grouped2['Date/Time'])

# Merge the two DataFrames on 'Date/Time'
df_msn_merged = pd.merge(df_msn_pct, df_msn_grouped2, on='Date/Time', how='inner')
df_msn_merged['Sold'] = df_msn_merged['Sold'].abs()

# Convert 'Date/Time' to datetime in both DataFrames
df_pnj_pct['Date/Time'] = pd.to_datetime(df_pnj_pct['Date/Time'])
df_pnj_grouped2['Date/Time'] = pd.to_datetime(df_pnj_grouped2['Date/Time'])

# Merge the two DataFrames on 'Date/Time'
df_pnj_merged = pd.merge(df_pnj_pct, df_pnj_grouped2, on='Date/Time', how='inner')
df_pnj_merged['Sold'] = df_pnj_merged['Sold'].abs()

# Convert 'Date/Time' to datetime in both DataFrames
df_vic_pct['Date/Time'] = pd.to_datetime(df_vic_pct['Date/Time'])
df_vic_grouped2['Date/Time'] = pd.to_datetime(df_vic_grouped2['Date/Time'])

# Merge the two DataFrames on 'Date/Time'
df_vic_merged = pd.merge(df_vic_pct, df_vic_grouped2, on='Date/Time', how='inner')
df_vic_merged['Sold'] = df_vic_merged['Sold'].abs()


**Bảng dữ liệu cho học máy bao gồm ngày, Thay đổi giá cổ phiếu ngày tiếp theo, Phần trăm thay đổi của khối lượng mua bán, Phần trăm thay đổi khối lượng mua, Phần trăm thay đổi khối lượng bán**

Bảng dữ liệu của FPT

In [350]:
# Calculate percentage change for 'Sell_Lastday' and 'Sold_Lastday'
df_fpt_merged['Pct_Sold_Change'] = df_fpt_merged['Sold'].pct_change()
df_fpt_merged['Pct_Bought_Change'] = df_fpt_merged['Bought'].pct_change()
df_fpt_merged['Pct_Change_Price'] = df_fpt_merged['Pct_Change_Price'].shift(-1)
df_fpt_merged = df_fpt_merged.rename(columns={'Pct_Change_Price': 'PriceChangeNextDay'})
df_fpt_merged = df_fpt_merged.drop(df_fpt_merged.index[0])
df_fpt_merged = df_fpt_merged.drop(columns=['Sold', 'Bought'])
df_fpt_merged

,Date/Time,PriceChangeNextDay,Pct_Change_Volume,Pct_Sold_Change,Pct_Bought_Change
1,2018-12-26,0.009712,2.926436,1.604264,8.420510
2,2018-12-27,0.009618,0.363530,0.094763,0.672271
3,2018-12-28,-0.023817,-0.148815,0.362887,-0.533626
4,2019-01-02,-0.009759,0.185836,-0.157541,0.940453
5,2019-01-03,0.007227,0.560812,0.844315,0.290317
...,...,...,...,...,...
494,2020-12-16,-0.010508,-0.251742,-0.141837,-0.376614
495,2020-12-17,0.007080,0.801276,0.681435,1.059800
496,2020-12-18,0.010545,-0.573565,-0.610301,-0.575886
497,2020-12-21,0.010435,0.505324,0.430945,0.715067


Bảng dữ liệu của MSN

In [ ]:
# Calculate percentage change for 'Sell_Lastday' and 'Sold_Lastday'
df_msn_merged['Pct_Sold_Change'] = df_msn_merged['Sold'].pct_change()
df_msn_merged['Pct_Bought_Change'] = df_msn_merged['Bought'].pct_change()
df_msn_merged['Pct_Change_Price'] = df_msn_merged['Pct_Change_Price'].shift(-1)
df_msn_merged = df_msn_merged.rename(columns={'Pct_Change_Price': 'PriceChangeNextDay'})
df_msn_merged = df_msn_merged.drop(df_msn_merged.index[0])
df_msn_merged = df_msn_merged.drop(columns=['Sold', 'Bought'])
df_msn_merged

,Date/Time,PriceChangeNextDay,Pct_Change_Volume,Pct_Sold_Change,Pct_Bought_Change
1,2017-12-26,-0.006623,0.946810,2.345967,0.263101
2,2017-12-27,0.009333,0.282901,0.300316,0.303211
3,2017-12-28,0.013210,-0.272030,-0.744361,0.281204
4,2017-12-29,0.043025,-0.398244,0.042267,-0.496324
5,2018-01-02,0.031250,1.353447,0.618173,1.804352
...,...,...,...,...,...
743,2020-12-16,0.005988,-0.306382,-0.002617,-0.480122
744,2020-12-17,0.001190,0.819055,-0.049807,1.823039
745,2020-12-18,-0.003567,0.201521,-0.036831,0.353458
746,2020-12-21,-0.011933,-0.416552,0.355150,-0.675952


Bảng dữ liệu của PNJ

In [ ]:
# Calculate percentage change for 'Sell_Lastday' and 'Sold_Lastday'
df_pnj_merged['Pct_Sold_Change'] = df_pnj_merged['Sold'].pct_change()
df_pnj_merged['Pct_Bought_Change'] = df_pnj_merged['Bought'].pct_change()
df_pnj_merged['Pct_Change_Price'] = df_pnj_merged['Pct_Change_Price'].shift(-1)
df_pnj_merged = df_pnj_merged.rename(columns={'Pct_Change_Price': 'PriceChangeNextDay'})
df_pnj_merged = df_pnj_merged.drop(df_pnj_merged.index[0])
df_pnj_merged = df_pnj_merged.drop(columns=['Sold', 'Bought'])
df_pnj_merged

,Date/Time,PriceChangeNextDay,Pct_Change_Volume,Pct_Sold_Change,Pct_Bought_Change
1,2018-03-01,-0.004249,1.514219,1.748848,1.274238
2,2018-03-02,0.012285,0.817269,1.021960,0.564217
3,2018-03-05,0.040496,-0.255862,-0.593798,0.284171
4,2018-03-06,-0.008717,0.480946,0.038548,0.704570
5,2018-03-07,0.019941,-0.253986,0.796733,-0.577583
...,...,...,...,...,...
704,2020-12-16,-0.023196,-0.266893,-0.246515,-0.283162
705,2020-12-17,0.019789,1.026910,1.594047,0.615623
706,2020-12-18,0.000000,-0.548088,-0.640704,-0.461779
707,2020-12-21,0.012937,0.160535,-0.079542,0.384022


Bảng dữ liệu của VIC

In [ ]:
# Calculate percentage change for 'Sell_Lastday' and 'Sold_Lastday'
df_vic_merged['Pct_Sold_Change'] = df_vic_merged['Sold'].pct_change()
df_vic_merged['Pct_Bought_Change'] = df_vic_merged['Bought'].pct_change()
df_vic_merged['Pct_Change_Price'] = df_vic_merged['Pct_Change_Price'].shift(-1)
df_vic_merged = df_vic_merged.rename(columns={'Pct_Change_Price': 'PriceChangeNextDay'})
df_vic_merged = df_vic_merged.drop(df_vic_merged.index[0])
df_vic_merged = df_vic_merged.drop(columns=['Sold', 'Bought'])
df_vic_merged

,Date/Time,PriceChangeNextDay,Pct_Change_Volume,Pct_Sold_Change,Pct_Bought_Change
1,2018-10-18,0.000000,-0.292489,-0.459786,0.012046
2,2018-10-19,0.000000,0.022017,0.501030,-0.443423
3,2018-10-22,-0.001009,-0.413749,-0.527636,-0.115310
4,2018-10-23,-0.002020,0.133760,-0.041931,0.379580
5,2018-10-24,-0.020243,0.119162,0.133767,0.104972
...,...,...,...,...,...
543,2020-12-16,-0.021495,-0.372812,-0.213073,-0.521259
544,2020-12-17,0.000000,0.788654,0.721856,1.126005
545,2020-12-18,0.016237,-0.001331,-0.571300,0.620334
546,2020-12-21,-0.006579,-0.565362,0.067488,-0.780650


Xây dựng mô hình học máy có giám sát LinearRegression để dự đoán mức độ thay đổi giá cổ phiếu ngày tiếp theo với các thông tin đầu vào là :
1.   Phần trăm thay đổi khối lượng giao dịch của mã cổ phiếu trong ngày
2.   Phần trăm thay đổi khối lượng giao dịch mua trong ngày
3.   Phần trăm thay đổi khối lượng giao dịch mua trong ngày


1. Dự đoán mức độ thay đổi của giá cổ phiếu FPT ngày tiếp theo.

In [ ]:
df_fpt_merged = df_fpt_merged.drop(columns=['Date/Time'])
df_fpt_predict = df_fpt_merged.iloc[-1:]
df_fpt_merged = df_fpt_merged.iloc[:-1]
# Split the data into training and test sets
X = df_fpt_merged.drop('PriceChangeNextDay', axis=1)
y = df_fpt_merged['PriceChangeNextDay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)
print(f'Độ sai lệch trung bình của mô hình là: {mse*100:.2f}%')
X_predict = df_fpt_predict.drop('PriceChangeNextDay', axis=1)
y_predict = model.predict(X_predict)

print(f'Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: {y_predict[0]*100:.3f}%')
# Get the last value of the 'Close' column
current_price = df_fpt_grouped['Close'].iloc[-1]

# Or you can use
# current_price = df_grouped['Close'].tail(1).values[0]

print(f'Giá cổ phiếu FPT hiện tại là: {current_price*1000:.0f} VNĐ')

# Calculate the next day's stock price
next_day_price = current_price * (1 + y_predict[0])

print(f'Giá cổ phiếu FPT được dự đoán trong ngày tiếp theo là: {next_day_price*1000:.0f} VNĐ')
print(f'Vậy giá cổ phiếu FPT đã biến động tăng: {(next_day_price - current_price)*1000:.0f} VNĐ')

Độ sai lệch trung bình của mô hình là: 0.02%
Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: 0.146%
Giá cổ phiếu FPT hiện tại là: 58100 VNĐ
Giá cổ phiếu FPT được dự đoán trong ngày tiếp theo là: 58185 VNĐ
Vậy giá cổ phiếu FPT đã biến động tăng: 85 VNĐ


2. Dự đoán mức độ thay đổi của giá cổ phiếu MSN ngày tiếp theo.

In [ ]:
df_msn_merged = df_msn_merged.drop(columns=['Date/Time'])
df_msn_predict = df_msn_merged.iloc[-1:]
df_msn_merged = df_msn_merged.iloc[:-1]
# Split the data into training and test sets
X = df_msn_merged.drop('PriceChangeNextDay', axis=1)
y = df_msn_merged['PriceChangeNextDay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)
print(f'Độ sai lệch trung bình của mô hình là: {mse*100:.2f}%')
X_predict = df_msn_predict.drop('PriceChangeNextDay', axis=1)
y_predict = model.predict(X_predict)

print(f'Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: {y_predict[0]*100:.3f}%')
# Get the last value of the 'Close' column
current_price = df_msn_grouped['Close'].iloc[-1]

# Or you can use
# current_price = df_grouped['Close'].tail(1).values[0]

print(f'Giá cổ phiếu MSN hiện tại là: {current_price*1000:.0f} VNĐ')

# Calculate the next day's stock price
next_day_price = current_price * (1 + y_predict[0])

print(f'Giá cổ phiếu MSN được dự đoán trong ngày tiếp theo là: {next_day_price*1000:.0f} VNĐ')
print(f'Vậy giá cổ phiếu MSN đã biến động tăng: {(next_day_price - current_price)*1000:.0f} VNĐ')

Độ sai lệch trung bình của mô hình là: 0.05%
Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: 0.002%
Giá cổ phiếu MSN hiện tại là: 82800 VNĐ
Giá cổ phiếu MSN được dự đoán trong ngày tiếp theo là: 82802 VNĐ
Vậy giá cổ phiếu MSN đã biến động tăng: 2 VNĐ


3. Dự đoán mức độ thay đổi của giá cổ phiếu PNJ ngày tiếp theo.

In [ ]:
df_pnj_merged = df_pnj_merged.drop(columns=['Date/Time'])
df_pnj_predict = df_pnj_merged.iloc[-1:]
df_pnj_merged = df_pnj_merged.iloc[:-1]
# Split the data into training and test sets
X = df_pnj_merged.drop('PriceChangeNextDay', axis=1)
y = df_pnj_merged['PriceChangeNextDay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)
print(f'Độ sai lệch trung bình của mô hình là: {mse*100:.2f}%')
X_predict = df_pnj_predict.drop('PriceChangeNextDay', axis=1)
y_predict = model.predict(X_predict)

print(f'Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: {y_predict[0]*100:.3f}%')
# Get the last value of the 'Close' column
current_price = df_pnj_grouped['Close'].iloc[-1]

# Or you can use
# current_price = df_grouped['Close'].tail(1).values[0]

print(f'Giá cổ phiếu PNJ hiện tại là: {current_price*1000:.0f} VNĐ')

# Calculate the next day's stock price
next_day_price = current_price * (1 + y_predict[0])

print(f'Giá cổ phiếu PNJ được dự đoán trong ngày tiếp theo là: {next_day_price*1000:.0f} VNĐ')
print(f'Vậy giá cổ phiếu PNJ đã biến động tăng: {(next_day_price - current_price)*1000:.0f} VNĐ')

Độ sai lệch trung bình của mô hình là: 0.06%
Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: 0.002%
Giá cổ phiếu PNJ hiện tại là: 78300 VNĐ
Giá cổ phiếu PNJ được dự đoán trong ngày tiếp theo là: 78302 VNĐ
Vậy giá cổ phiếu PNJ đã biến động tăng: 2 VNĐ


4. Dự đoán mức độ thay đổi của giá cổ phiếu VIC ngày tiếp theo.

In [ ]:
df_vic_merged = df_vic_merged.drop(columns=['Date/Time'])
df_vic_predict = df_vic_merged.iloc[-1:]
df_vic_merged = df_vic_merged.iloc[:-1]
# Split the data into training and test sets
X = df_vic_merged.drop('PriceChangeNextDay', axis=1)
y = df_vic_merged['PriceChangeNextDay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)
print(f'Độ sai lệch trung bình của mô hình là: {mse*100:.2f}%')
X_predict = df_vic_predict.drop('PriceChangeNextDay', axis=1)
y_predict = model.predict(X_predict)

print(f'Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: {y_predict[0]*100:.3f}%')
# Get the last value of the 'Close' column
current_price = df_vic_grouped['Close'].iloc[-1]

# Or you can use
# current_price = df_grouped['Close'].tail(1).values[0]

print(f'Giá cổ phiếu VIC hiện tại là: {current_price*1000:.0f} VNĐ')

# Calculate the next day's stock price
next_day_price = current_price * (1 + y_predict[0])

print(f'Giá cổ phiếu VIC được dự đoán trong ngày tiếp theo là: {next_day_price*1000:.0f} VNĐ')
print(f'Vậy giá cổ phiếu VIC đã biến động tăng: {(next_day_price - current_price)*1000:.0f} VNĐ')

Độ sai lệch trung bình của mô hình là: 0.02%
Dự đoán độ thay đổi giá cổ phiếu ngày tiếp theo là: 0.079%
Giá cổ phiếu VIC hiện tại là: 105700 VNĐ
Giá cổ phiếu VIC được dự đoán trong ngày tiếp theo là: 105783 VNĐ
Vậy giá cổ phiếu VIC đã biến động tăng: 83 VNĐ
